In [32]:
import os
import json

file_path = '/tf/notebooks/NFS_Dataset/YOLO_with_ReID/results/json_group1(only day)_Done/'
dict_data = []

for fp in os.listdir(file_path) :
    json_path = os.path.join(file_path, fp)
    
    if os.path.isdir(json_path): # 디렉토리는 넘기기
        continue
        
    with open(json_path, "r") as f:
        data = json.load(f)
        label = data['label']
        videoname = os.path.splitext(fp)[0] + '.mp4'
        id_li = []
        for frame in data['frames'] :
            if frame['frame_id'] % 30 == 0 :
                id_li.append(frame['objects'])
        frame_num = 0
        text = '<video>\nCould you please tell me if the video is of assault and if so, what happened?'
        for i in id_li :
            for j in i :
                text += f"frame{frame_num} id : {j['id']}"
            frame_num += 30
        print(text)
        
    # data = {
    #     "video": ff,
    #     "conversations": [
    #         {"from": "human", "value": human_value},
    #         {"from": "gpt", "value": label}
    #     ]
    # }

frame1 id : 1frame1 id : 2frame31 id : 2frame61 id : 1frame61 id : 2frame91 id : 1frame91 id : 2frame121 id : 1frame121 id : 2
frame1 id : 1frame1 id : 2frame31 id : 1frame31 id : 2frame61 id : 1frame91 id : 1frame91 id : 2frame121 id : 1frame121 id : 2
frame1 id : 1frame1 id : 2frame31 id : 1frame31 id : 2frame31 id : 3frame61 id : 1frame61 id : 2frame91 id : 1frame91 id : 2frame91 id : 3frame121 id : 1frame121 id : 2frame121 id : 3frame121 id : 4
frame1 id : 1frame1 id : 2frame31 id : 1frame31 id : 2frame61 id : 1frame61 id : 2frame91 id : 1frame91 id : 2frame121 id : 2frame121 id : 1
frame1 id : 1frame1 id : 2frame31 id : 1frame31 id : 2frame61 id : 1frame61 id : 2frame91 id : 1frame91 id : 2frame121 id : 1frame121 id : 2
frame1 id : 1frame1 id : 2frame1 id : 3frame31 id : 1frame31 id : 2frame31 id : 3frame61 id : 1frame61 id : 2frame61 id : 3frame91 id : 1frame91 id : 2frame91 id : 3frame121 id : 1frame121 id : 2frame121 id : 3
frame1 id : 1frame1 id : 2frame1 id : 3frame1 id : 4fr

In [37]:
import os
import json
import numpy as np

input_folder = "/tf/notebooks/NFS_Dataset/YOLO_with_ReID/results/json_group1(only day)_Done"
output_file = "./assault_train.json"
common_question = "<video>\nCould you please tell me if the video is of assault and if so, what happened?"

merged_data = []  # root 리스트 대신 바로 리스트로

for filename in os.listdir(input_folder):
    if filename.endswith(".json"):
        file_path = os.path.join(input_folder, filename)
        with open(file_path, "r") as f:
            data = json.load(f)
        
        # json 파일 이름에서 .json 제거하고 .mp4로 변경
        video_name = os.path.splitext(filename)[0] + "_tracked.mp4"
        label = data.get("label")
        frames = data.get("frames", [])
        total_frames = len(frames)
        
        selected_idx = np.arange(1, total_frames, 30)
        
        # 프레임별로 묶어서 요약
        frame_summary = []
        for idx in selected_idx:
            if idx < total_frames:
                frame = frames[idx]
                frame_id = frame.get("frame_id")
                obj_summaries = []
                for obj in frame.get("objects", []):
                    obj_summaries.append(f"id:{obj['id']} bbox:{obj['bbox']}")
                if obj_summaries:
                    combined = ", ".join(obj_summaries)
                    frame_summary.append(f"frame{frame_id}: persons: [{combined}]")

        object_summary_str = "\n".join(frame_summary)
        human_value = f"{common_question}\n{object_summary_str}"

        entry = {
            "video": video_name,
            "conversations": [
                {"from": "human", "value": human_value},
                {"from": "gpt", "value": label}
            ]
        }

        merged_data.append(entry)  # root 대신 바로 리스트에 추가

In [38]:
merged_data[0]

{'video': '12-1_cam02_assault01_place09_day_spring_person2_kicking_seg1_tracked.mp4',
 'conversations': [{'from': 'human',
   'value': '<video>\nCould you please tell me if the video is of assault and if so, what happened?\nframe1: persons: [id:1 bbox:[555, 254, 600, 484], id:2 bbox:[585, 270, 640, 529]]\nframe31: persons: [id:2 bbox:[507, 260, 593, 555], id:1 bbox:[567, 267, 637, 497]]\nframe61: persons: [id:1 bbox:[570, 273, 627, 509], id:2 bbox:[360, 450, 517, 607]]\nframe91: persons: [id:1 bbox:[568, 282, 641, 543], id:2 bbox:[361, 480, 477, 619]]\nframe121: persons: [id:1 bbox:[562, 278, 635, 550], id:2 bbox:[362, 479, 477, 622]]'},
  {'from': 'gpt',
   'value': 'In an assault situation, ID1 who is wearing checkered shirt and blue pants is punching ID2 who is wearing checkered shirt and black pants.'}]}

In [39]:
with open(output_file, "w") as f_out:
    json.dump(merged_data, f_out, indent=2)

print(f"Saved merged file to {output_file}")

Saved merged file to ./assault_train.json
